In [1]:
!pip install mysql-connector-python

In [5]:
import mysql.connector
import pandas as pd

# ENSURE DATABASE CONNECTION AND CREATION
def create_database():
    """Ensures that the MySQL database exists."""
    # CREATE CONNECTION TO MYSQL SERVER
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="353txRQ8"
    )
    # CREATE CURSOR OBJECT TO INTERACT WITH THE DATABASE
    cursor = conn.cursor()
    # CREATE DATABASE IF IT DOESN'T EXIST
    cursor.execute("CREATE DATABASE IF NOT EXISTS data_engineering_books")
    # COMMIT THE TRANSACTION
    conn.commit()
    # CLOSE CURSOR AND CONNECTION
    cursor.close()
    conn.close()

# ENSURE TABLE CREATION
def create_table():
    """Ensures that the 'books' table exists with the correct schema."""
    # CREATE CONNECTION TO MYSQL SERVER WITH DATABASE SELECTION
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="353txRQ8",
        database="data_engineering_books"
    )
    # CREATE CURSOR OBJECT TO INTERACT WITH THE DATABASE
    cursor = conn.cursor()
    
    # CREATE TABLE IF IT DOESN'T EXIST
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INT AUTO_INCREMENT PRIMARY KEY,  # PRIMARY KEY AND AUTO INCREMENT FOR ID COLUMN
            title VARCHAR(255),  # BOOK TITLE COLUMN
            author VARCHAR(255),  # BOOK AUTHOR COLUMN
            publication_date VARCHAR(50),  # PUBLICATION DATE COLUMN
            rating VARCHAR(10),  # BOOK RATING COLUMN
            price VARCHAR(50)  # BOOK PRICE COLUMN
        )
    """)
    
    # COMMIT THE TRANSACTION TO SAVE CHANGES
    conn.commit()
    # CLOSE CURSOR AND CONNECTION
    cursor.close()
    conn.close()

# LOAD DATA FROM THE CSV FILE AND HANDLE NaN VALUES
def load_data():
    """Loads scraped data from CSV, handles missing values, and inserts into MySQL."""
    # READ DATA FROM CSV FILE INTO A PANDAS DATAFRAME
    df = pd.read_csv("amazon_data_engineering_books.csv")

    # FILL NaN VALUES WITH DEFAULT TEXT FOR TEXT FIELDS AND "NOT AVAILABLE" FOR NUMERIC FIELDS
    df.fillna({
        'Title': 'Unknown',  # fills empty values in 'Title' with 'Unknown'
        'Author': 'Unknown', 
        'Publication Date': 'Not Available',
        'Rating': 'No Rating', 
        'Price': 'Not Available'  
    }, inplace=True)

    conn = None
    cursor = None

    try:
        # CREATE CONNECTION TO MYSQL DATABASE
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="353txRQ8",
            database="data_engineering_books"
        )
        # CREATE CURSOR OBJECT TO INTERACT WITH THE DATABASE
        cursor = conn.cursor()

        # INSERT DATA INTO THE 'books' TABLE
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO books (title, author, publication_date, rating, price)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['Title'], row['Author'], row['Publication Date'], row['Rating'], row['Price']))
        
        # COMMIT THE TRANSACTION TO SAVE CHANGES
        conn.commit()
        print("Data successfully inserted into MySQL!")

    except mysql.connector.Error as db_error:
        # PRINT ERROR MESSAGE IN CASE OF A DATABASE ERROR
        print(f"Database error: {db_error}")

    finally:
        # CLOSE CURSOR AND CONNECTION AFTER USE
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# EXTRACT AND SORT DATA
def query_books():
    """Executes the SQL query to extract and sort books based on rating."""
    # CREATE CONNECTION TO MYSQL DATABASE
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="353txRQ8",
        database="data_engineering_books"
    )
    # CREATE CURSOR OBJECT TO INTERACT WITH THE DATABASE
    cursor = conn.cursor()

    # SELECT TITLE, AUTHOR, AND RATING COLUMNS, ORDERING BY RATING IN DESCENDING ORDER
    query = """
        SELECT title, author, rating
        FROM books
        ORDER BY rating DESC;  # ORDER BY RATING DESCENDING
    """
    cursor.execute(query)
    results = cursor.fetchall()

    # CREATE A PANDAS DATAFRAME FROM THE QUERY RESULT
    df_results1 = pd.DataFrame(results, columns=["Title", "Author", "Price", "Publication Date"])
    
    print("\nTop-rated Data Engineering Books:\n")
    print(df_results1)

    # SELECT TITLE, PUBLICATION DATE, AND PRICE COLUMNS, ORDERING BY PUBLICATION DATE IN ASCENDING ORDER
    query2 = """
        SELECT title, publication_date, price
        FROM books
        ORDER BY publication_date ASC;  # ORDER BY PUBLICATION DATE ASCENDING
    """
    cursor.execute(query2)
    results2 = cursor.fetchall()

    # CREATE A PANDAS DATAFRAME FROM THE SECOND QUERY RESULT
    df_results2 = pd.DataFrame(results2, columns=["Title", "Publication Date", "Price"])
    print("\nBooks Sorted by Publication Date (Oldest to Newest):\n")
    print(df_results2)

    # CLOSE CURSOR AND CONNECTION
    cursor.close()
    conn.close()

# EXECUTE FUNCTIONS
create_database()  # CREATE DATABASE IF IT DOESN'T EXIST
create_table()  # CREATE THE 'books' TABLE IF IT DOESN'T EXIST
load_data()  # LOAD DATA INTO THE DATABASE FROM CSV
query_books()  # QUERY AND DISPLAY THE RESULTS

FileNotFoundError: [Errno 2] No such file or directory: 'amazon_data_engineering_books.csv'